In [4]:
### expriry for IH IF IC
### expriry for IH IF IC
import pandas as pd
import TSLPy3
import datetime

monList=['201707','201708','201709','201710','201711','201712',
         '201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
         '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
         '202001','202002','202003','202004','202005','202006','202007','202008','202009', '202010', '202011', '202012',
         '202101', '202103', '202106']   
IHList = ['IH' + mon[2:] for mon in monList]
contr_df = []
for contr_name in IHList:
    tsstr="""
           BegT :=%s;
           EndT :=%s + 0.99;
           setSysParam(pn_stock(),'%s');
           n:= tradedays(BegT,EndT);
           df_day:= NDay(n,'firstDate',sp_time(),'expDate',base(703024));
           return df_day;
           """%('20170101T','20201231T',contr_name)
    df_day = pd.DataFrame(TSLPy3.RemoteExecute(tsstr,{})[1])
    df_day.columns = list(pd.Series(df_day.columns).str.decode('GBK'))
    df_day['firstDate'] = df_day.firstDate.map(lambda x: int((datetime.datetime(1899,12,30) + datetime.timedelta(x)).strftime('%Y%m%d'))) 
    df_day['contr_name'] = 'CFFEX' + contr_name[2:]
    contr_df += [df_day.groupby(['contr_name']).min().reset_index()]
contr_df = pd.concat(contr_df).reset_index(drop=True)
contr_df.to_csv(r'A:\cffex\expiryTable_20201203.csv', index=False)

In [6]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

readPath = r'\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip'
path1 = np.array(glob.glob(os.path.join(readPath,'IC_***')))
path1 = path1[(np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in path1]) >= '202010') & np.array([len(i)==71 for i in path1])]
s = pd.DataFrame()
for p in path1:
    readPath = p
    dataPathLs = np.array(glob.glob(os.path.join(readPath,'FFL2_TAQ_***')))
    IC_list = np.unique([os.path.basename(x).split('_')[2] for x in dataPathLs])
    ll = pd.DataFrame()
    for j in dataPathLs:
        df = pd.read_csv(j)
        df = df[["TRADINGDATE", "TRADINGTIME", "SYMBOL", "TOTALVOLUME", "TOTALAMOUNT", "LASTPRICE", "BUYPRICE01", "BUYPRICE02",
         "BUYPRICE03", "BUYPRICE04","BUYPRICE05","BUYVOLUME01", "BUYVOLUME02", "BUYVOLUME03","BUYVOLUME04", "BUYVOLUME05",
         "SELLPRICE01", "SELLPRICE02", "SELLPRICE03", "SELLPRICE04", "SELLPRICE05", "SELLVOLUME01",  "SELLVOLUME02", 
         "SELLVOLUME03","SELLVOLUME04","SELLVOLUME05","OPENPRICE", "SETTLEPRICE"]]
        df["TRADINGTIME"] = pd.to_datetime(df["TRADINGTIME"])
        df = df.rename(columns={"TRADINGDATE":"date", "TRADINGTIME":"time", "SYMBOL":"ID", "TOTALVOLUME":"cum_volume", "TOTALAMOUNT":"cum_amount",
                       "LASTPRICE":"close", "BUYPRICE01":"bid1p", "BUYPRICE02":"bid2p", "BUYPRICE03":"bid3p", "BUYPRICE04":"bid4p",
                       "BUYPRICE05":"bid5p", "BUYVOLUME01":"bid1q", "BUYVOLUME02":"bid2q", "BUYVOLUME03":"bid3q", "BUYVOLUME04":"bid4q",
                       "BUYVOLUME05":"bid5q", "SELLPRICE01":"ask1p", "SELLPRICE02":"ask2p", "SELLPRICE03":"ask3p", "SELLPRICE04":"ask4p",
                       "SELLPRICE05":"ask5p", "SELLVOLUME01": "ask1q", "SELLVOLUME02": "ask2q", "SELLVOLUME03": "ask3q", 
                       "SELLVOLUME04": "ask4q", "SELLVOLUME05": "ask5q", "OPENPRICE":"openPrice", "SETTLEPRICE":"settlePrice"})
        for i in ["cum_volume", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "ask1q", "ask2q", "ask3q","ask4q", "ask5q"]:
            df[i] = df[i].astype(int)
        ll = pd.concat([ll, df])
    r = ll[~ll["ID"].isin(["ICM01", "ICM02"])].groupby("date")["ID"].unique()
    s1 = r.apply(lambda x: x[0]).reset_index()
    s1["contractStatus"] = 1
    s2 = r.apply(lambda x: x[1]).reset_index()
    s2["contractStatus"] = 2
    s3 = r.apply(lambda x: x[2]).reset_index()
    s3["contractStatus"] = 3
    s4 = r.apply(lambda x: x[3]).reset_index()
    s4["contractStatus"] = 4
    s = pd.concat([s, s1, s2, s3, s4])
print(s)

        date      ID  contractStatus
0   20201009  IC2010               1
1   20201012  IC2010               1
2   20201013  IC2010               1
3   20201014  IC2010               1
4   20201015  IC2010               1
..       ...     ...             ...
16  20201124  IC2106               4
17  20201125  IC2106               4
18  20201126  IC2106               4
19  20201127  IC2106               4
20  20201130  IC2106               4

[148 rows x 3 columns]


In [8]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

kk = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\cffex\expiryTable_20201203.csv')
kk["ID"] = kk["contr_name"].str[5:].astype(int)
kk["expDate"] = pd.to_datetime(kk["expDate"].astype(str), format='%Y%m%d')
readPath = r'\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC***'
# part 1
readPath = np.array(glob.glob(readPath))
readPath = readPath[(np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in readPath]) >= '202010')]

dataPathLs = []
for i in readPath:
    if len(glob.glob(os.path.join(i,'FFL2_TAQ_***'))) != 0:
        for j in glob.glob(os.path.join(i,'FFL2_TAQ_***')):
            dataPathLs.append(j)
# part 2
# dataPathLs = np.array(glob.glob(os.path.join(readPath,'FFL2_TAQ_***')))
IC_list = np.unique([os.path.basename(i).split('_')[2] for i in dataPathLs])
IC_list = np.delete(IC_list, np.argwhere(IC_list=='ICM01'))
IC_list = np.delete(IC_list, np.argwhere(IC_list=='ICM02'))
# IC_list = np.delete(IC_list, np.argwhere(IC_list=='IC2007'))
print(IC_list)

for l in IC_list:
    path = []
    df1 = pd.DataFrame()
    for i in dataPathLs:
        if os.path.basename(i).split('_')[2] == l:
            path.append(i)
    for j in path:
        df = pd.read_csv(j)
        df = df[["TRADINGDATE", "TRADINGTIME", "SYMBOL", "TOTALVOLUME", "TOTALAMOUNT", "LASTPRICE", "BUYPRICE01", "BUYPRICE02",
         "BUYPRICE03", "BUYPRICE04","BUYPRICE05","BUYVOLUME01", "BUYVOLUME02", "BUYVOLUME03","BUYVOLUME04", "BUYVOLUME05",
         "SELLPRICE01", "SELLPRICE02", "SELLPRICE03", "SELLPRICE04", "SELLPRICE05", "SELLVOLUME01",  "SELLVOLUME02", 
         "SELLVOLUME03","SELLVOLUME04","SELLVOLUME05","OPENPRICE", "SETTLEPRICE"]]
        df["TRADINGTIME"] = pd.to_datetime(df["TRADINGTIME"])
        df = df.rename(columns={"TRADINGDATE":"date", "TRADINGTIME":"time", "SYMBOL":"ID", "TOTALVOLUME":"cum_volume", "TOTALAMOUNT":"cum_amount",
                       "LASTPRICE":"close", "BUYPRICE01":"bid1p", "BUYPRICE02":"bid2p", "BUYPRICE03":"bid3p", "BUYPRICE04":"bid4p",
                       "BUYPRICE05":"bid5p", "BUYVOLUME01":"bid1q", "BUYVOLUME02":"bid2q", "BUYVOLUME03":"bid3q", "BUYVOLUME04":"bid4q",
                       "BUYVOLUME05":"bid5q", "SELLPRICE01":"ask1p", "SELLPRICE02":"ask2p", "SELLPRICE03":"ask3p", "SELLPRICE04":"ask4p",
                       "SELLPRICE05":"ask5p", "SELLVOLUME01": "ask1q", "SELLVOLUME02": "ask2q", "SELLVOLUME03": "ask3q", 
                       "SELLVOLUME04": "ask4q", "SELLVOLUME05": "ask5q", "OPENPRICE":"openPrice", "SETTLEPRICE":"settlePrice"})
        for i in ["cum_volume", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "ask1q", "ask2q", "ask3q","ask4q", "ask5q"]:
            df[i] = df[i].astype(int)
        df1 = pd.concat([df1, df]).reset_index(drop=True)
        print(j)
        print(df1.shape[0])
    
    d = kk[kk["ID"] == int(df1["ID"].iloc[0][2:])]["expDate"].values[0]
    df1["dayToExpiry"] = (d - pd.to_datetime(df1["date"].astype(str), format='%Y%m%d')).apply(lambda x: x.days)
    df1 = pd.merge(df1, s, on=["date", "ID"])
    print(df1.shape[0])
    try:
        ndf = pd.read_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')
        print(ndf.shape[0])
        print(ndf['date'].max())
        ndf = ndf[ndf['date'] < df1['date'].min()]
        assert(ndf['date'].max() < df1['date'].min())
        df1 = pd.concat([ndf, df1])
        print(df1.shape[0])
        print(df1["date"].min())
        print(df1["date"].max())
        df1.to_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')
    except:
        print('new contract')
        print(df1["date"].min())
        print(df1["date"].max())
        df1.to_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')



['IC2010' 'IC2011' 'IC2012' 'IC2101' 'IC2103' 'IC2106']
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202010\FFL2_TAQ_IC2010_202010.csv
154865
154865
817971
20201016
817971
20200824
20201016
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202010\FFL2_TAQ_IC2011_202010.csv
419277
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202011\FFL2_TAQ_IC2011_202011.csv
824553
824553
402302
20201022
975391
20200921
20201120
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202010\FFL2_TAQ_IC2012_202010.csv
389300
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202011\FFL2_TAQ_IC2012_202011.csv
944179
944179
2595813
20201022
3300907
20200420
20201130
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202011\FFL2_TAQ_IC2101_202011.csv
91293
91293
new contract
20201123
20201130
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\ICdata_zip\IC_202010\FFL2_TAQ_IC2103_202010.csv
317006
\\192.168.10.30\Kevi

In [9]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

readPath = r'\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip'
path1 = np.array(glob.glob(os.path.join(readPath,'IF_***')))
path1 = path1[(np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in path1]) >= '202010') & np.array([len(i)==71 for i in path1])]
s = pd.DataFrame()
for p in path1:
    readPath = p
    dataPathLs = np.array(glob.glob(os.path.join(readPath,'FFL2_TAQ_***')))
    IC_list = np.unique([os.path.basename(x).split('_')[2] for x in dataPathLs])
    ll = pd.DataFrame()
    for j in dataPathLs:
        df = pd.read_csv(j)
        df = df[["TRADINGDATE", "TRADINGTIME", "SYMBOL", "TOTALVOLUME", "TOTALAMOUNT", "LASTPRICE", "BUYPRICE01", "BUYPRICE02",
         "BUYPRICE03", "BUYPRICE04","BUYPRICE05","BUYVOLUME01", "BUYVOLUME02", "BUYVOLUME03","BUYVOLUME04", "BUYVOLUME05",
         "SELLPRICE01", "SELLPRICE02", "SELLPRICE03", "SELLPRICE04", "SELLPRICE05", "SELLVOLUME01",  "SELLVOLUME02", 
         "SELLVOLUME03","SELLVOLUME04","SELLVOLUME05","OPENPRICE", "SETTLEPRICE"]]
        df["TRADINGTIME"] = pd.to_datetime(df["TRADINGTIME"])
        df = df.rename(columns={"TRADINGDATE":"date", "TRADINGTIME":"time", "SYMBOL":"ID", "TOTALVOLUME":"cum_volume", "TOTALAMOUNT":"cum_amount",
                       "LASTPRICE":"close", "BUYPRICE01":"bid1p", "BUYPRICE02":"bid2p", "BUYPRICE03":"bid3p", "BUYPRICE04":"bid4p",
                       "BUYPRICE05":"bid5p", "BUYVOLUME01":"bid1q", "BUYVOLUME02":"bid2q", "BUYVOLUME03":"bid3q", "BUYVOLUME04":"bid4q",
                       "BUYVOLUME05":"bid5q", "SELLPRICE01":"ask1p", "SELLPRICE02":"ask2p", "SELLPRICE03":"ask3p", "SELLPRICE04":"ask4p",
                       "SELLPRICE05":"ask5p", "SELLVOLUME01": "ask1q", "SELLVOLUME02": "ask2q", "SELLVOLUME03": "ask3q", 
                       "SELLVOLUME04": "ask4q", "SELLVOLUME05": "ask5q", "OPENPRICE":"openPrice", "SETTLEPRICE":"settlePrice"})
        for i in ["cum_volume", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "ask1q", "ask2q", "ask3q","ask4q", "ask5q"]:
            df[i] = df[i].astype(int)
        ll = pd.concat([ll, df])
    r = ll[~ll["ID"].isin(["IFM01", "IFM02"])].groupby("date")["ID"].unique()
    s1 = r.apply(lambda x: x[0]).reset_index()
    s1["contractStatus"] = 1
    s2 = r.apply(lambda x: x[1]).reset_index()
    s2["contractStatus"] = 2
    s3 = r.apply(lambda x: x[2]).reset_index()
    s3["contractStatus"] = 3
    s4 = r.apply(lambda x: x[3]).reset_index()
    s4["contractStatus"] = 4
    s = pd.concat([s, s1, s2, s3, s4])
print(s)

        date      ID  contractStatus
0   20201009  IF2010               1
1   20201012  IF2010               1
2   20201013  IF2010               1
3   20201014  IF2010               1
4   20201015  IF2010               1
..       ...     ...             ...
16  20201124  IF2106               4
17  20201125  IF2106               4
18  20201126  IF2106               4
19  20201127  IF2106               4
20  20201130  IF2106               4

[148 rows x 3 columns]


In [10]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF***'

kk = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\cffex\expiryTable_20201203.csv')
kk["ID"] = kk["contr_name"].str[5:].astype(int)
kk["expDate"] = pd.to_datetime(kk["expDate"].astype(str), format='%Y%m%d')

# part 1
readPath = np.array(glob.glob(readPath))
readPath = readPath[(np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in readPath]) >= '202010')]

dataPathLs = []
for i in readPath:
    if len(glob.glob(os.path.join(i,'FFL2_TAQ_***'))) != 0:
        for j in glob.glob(os.path.join(i,'FFL2_TAQ_***')):
            dataPathLs.append(j)
# part 2
# dataPathLs = np.array(glob.glob(os.path.join(readPath,'FFL2_TAQ_***')))
IF_list = np.unique([os.path.basename(i).split('_')[2] for i in dataPathLs])
IF_list = np.delete(IF_list, np.argwhere(IF_list=='IFM01'))
IF_list = np.delete(IF_list, np.argwhere(IF_list=='IFM02'))
# IF_list = np.delete(IF_list, np.argwhere(IC_list=='IF2007'))
print(IF_list)

for l in IF_list:
    path = []
    df1 = pd.DataFrame()
    for i in dataPathLs:
        if os.path.basename(i).split('_')[2] == l:
            path.append(i)
    for j in path:
        df = pd.read_csv(j)
        df = df[["TRADINGDATE", "TRADINGTIME", "SYMBOL", "TOTALVOLUME", "TOTALAMOUNT", "LASTPRICE", "BUYPRICE01", "BUYPRICE02",
         "BUYPRICE03", "BUYPRICE04","BUYPRICE05","BUYVOLUME01", "BUYVOLUME02", "BUYVOLUME03","BUYVOLUME04", "BUYVOLUME05",
         "SELLPRICE01", "SELLPRICE02", "SELLPRICE03", "SELLPRICE04", "SELLPRICE05", "SELLVOLUME01",  "SELLVOLUME02", 
         "SELLVOLUME03","SELLVOLUME04","SELLVOLUME05","OPENPRICE", "SETTLEPRICE"]]
        df["TRADINGTIME"] = pd.to_datetime(df["TRADINGTIME"])
        df = df.rename(columns={"TRADINGDATE":"date", "TRADINGTIME":"time", "SYMBOL":"ID", "TOTALVOLUME":"cum_volume", "TOTALAMOUNT":"cum_amount",
                       "LASTPRICE":"close", "BUYPRICE01":"bid1p", "BUYPRICE02":"bid2p", "BUYPRICE03":"bid3p", "BUYPRICE04":"bid4p",
                       "BUYPRICE05":"bid5p", "BUYVOLUME01":"bid1q", "BUYVOLUME02":"bid2q", "BUYVOLUME03":"bid3q", "BUYVOLUME04":"bid4q",
                       "BUYVOLUME05":"bid5q", "SELLPRICE01":"ask1p", "SELLPRICE02":"ask2p", "SELLPRICE03":"ask3p", "SELLPRICE04":"ask4p",
                       "SELLPRICE05":"ask5p", "SELLVOLUME01": "ask1q", "SELLVOLUME02": "ask2q", "SELLVOLUME03": "ask3q", 
                       "SELLVOLUME04": "ask4q", "SELLVOLUME05": "ask5q", "OPENPRICE":"openPrice", "SETTLEPRICE":"settlePrice"})
        for i in ["cum_volume", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "ask1q", "ask2q", "ask3q","ask4q", "ask5q"]:
            df[i] = df[i].astype(int)
        df1 = pd.concat([df1, df]).reset_index(drop=True)
        print(j)
        print(df1.shape[0])
    
    d = kk[kk["ID"] == int(df1["ID"].iloc[0][2:])]["expDate"].values[0]
    df1["dayToExpiry"] = (d - pd.to_datetime(df1["date"].astype(str), format='%Y%m%d')).apply(lambda x: x.days)
    df1 = pd.merge(df1, s, on=["date", "ID"])
    print(df1.shape[0])
    try:
        ndf = pd.read_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')
        print(ndf.shape[0])
        print(ndf['date'].max())
        ndf = ndf[ndf['date'] < df1['date'].min()]
        assert(ndf['date'].max() < df1['date'].min())
        print(ndf.shape[0])
        df1 = pd.concat([ndf, df1])
        print(df1.shape[0])
        print(df1["date"].min())
        print(df1["date"].max())
        df1.to_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')
    except:
        print('new contract')
        print(df1["date"].min())
        print(df1["date"].max())
        df1.to_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')

['IF2010' 'IF2011' 'IF2012' 'IF2101' 'IF2103' 'IF2106']
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202010\FFL2_TAQ_IF2010_202010.csv
154404
154404
792972
20201016
638568
792972
20200824
20201016
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202010\FFL2_TAQ_IF2011_202010.csv
410125
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202011\FFL2_TAQ_IF2011_202011.csv
815088
815088
362944
20201022
120560
935648
20200921
20201120
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202010\FFL2_TAQ_IF2012_202010.csv
375834
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202011\FFL2_TAQ_IF2012_202011.csv
926216
926216
2295421
20201022
2065984
2992200
20200420
20201130
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202011\FFL2_TAQ_IF2101_202011.csv
91161
91161
new contract
20201123
20201130
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IFdata_zip\IF_202010\FFL2_TAQ_IF2103_202010.csv
27594

In [11]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip'
path1 = np.array(glob.glob(os.path.join(readPath,'IH_***')))
path1 = path1[(np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in path1]) >= '202010') & np.array([len(i)==71 for i in path1])]
s = pd.DataFrame()
for p in path1:
    readPath = p
    dataPathLs = np.array(glob.glob(os.path.join(readPath,'FFL2_TAQ_***')))
    IC_list = np.unique([os.path.basename(x).split('_')[2] for x in dataPathLs])
    ll = pd.DataFrame()
    for j in dataPathLs:
        df = pd.read_csv(j)
        df = df[["TRADINGDATE", "TRADINGTIME", "SYMBOL", "TOTALVOLUME", "TOTALAMOUNT", "LASTPRICE", "BUYPRICE01", "BUYPRICE02",
         "BUYPRICE03", "BUYPRICE04","BUYPRICE05","BUYVOLUME01", "BUYVOLUME02", "BUYVOLUME03","BUYVOLUME04", "BUYVOLUME05",
         "SELLPRICE01", "SELLPRICE02", "SELLPRICE03", "SELLPRICE04", "SELLPRICE05", "SELLVOLUME01",  "SELLVOLUME02", 
         "SELLVOLUME03","SELLVOLUME04","SELLVOLUME05","OPENPRICE", "SETTLEPRICE"]]
        df["TRADINGTIME"] = pd.to_datetime(df["TRADINGTIME"])
        df = df.rename(columns={"TRADINGDATE":"date", "TRADINGTIME":"time", "SYMBOL":"ID", "TOTALVOLUME":"cum_volume", "TOTALAMOUNT":"cum_amount",
                       "LASTPRICE":"close", "BUYPRICE01":"bid1p", "BUYPRICE02":"bid2p", "BUYPRICE03":"bid3p", "BUYPRICE04":"bid4p",
                       "BUYPRICE05":"bid5p", "BUYVOLUME01":"bid1q", "BUYVOLUME02":"bid2q", "BUYVOLUME03":"bid3q", "BUYVOLUME04":"bid4q",
                       "BUYVOLUME05":"bid5q", "SELLPRICE01":"ask1p", "SELLPRICE02":"ask2p", "SELLPRICE03":"ask3p", "SELLPRICE04":"ask4p",
                       "SELLPRICE05":"ask5p", "SELLVOLUME01": "ask1q", "SELLVOLUME02": "ask2q", "SELLVOLUME03": "ask3q", 
                       "SELLVOLUME04": "ask4q", "SELLVOLUME05": "ask5q", "OPENPRICE":"openPrice", "SETTLEPRICE":"settlePrice"})
        for i in ["cum_volume", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "ask1q", "ask2q", "ask3q","ask4q", "ask5q"]:
            df[i] = df[i].astype(int)
        ll = pd.concat([ll, df])
    r = ll[~ll["ID"].isin(["IHM01", "IHM02"])].groupby("date")["ID"].unique()
    s1 = r.apply(lambda x: x[0]).reset_index()
    s1["contractStatus"] = 1
    s2 = r.apply(lambda x: x[1]).reset_index()
    s2["contractStatus"] = 2
    s3 = r.apply(lambda x: x[2]).reset_index()
    s3["contractStatus"] = 3
    s4 = r.apply(lambda x: x[3]).reset_index()
    s4["contractStatus"] = 4
    s = pd.concat([s, s1, s2, s3, s4])
print(s)

        date      ID  contractStatus
0   20201009  IH2010               1
1   20201012  IH2010               1
2   20201013  IH2010               1
3   20201014  IH2010               1
4   20201015  IH2010               1
..       ...     ...             ...
16  20201124  IH2106               4
17  20201125  IH2106               4
18  20201126  IH2106               4
19  20201127  IH2106               4
20  20201130  IH2106               4

[148 rows x 3 columns]


In [12]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH***'
# part 1
readPath = np.array(glob.glob(readPath))
readPath = readPath[(np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in readPath]) >= '202010')]

dataPathLs = []
for i in readPath:
    if len(glob.glob(os.path.join(i,'FFL2_TAQ_***'))) != 0:
        for j in glob.glob(os.path.join(i,'FFL2_TAQ_***')):
            dataPathLs.append(j)
# part 2
# dataPathLs = np.array(glob.glob(os.path.join(readPath,'FFL2_TAQ_***')))
IH_list = np.unique([os.path.basename(i).split('_')[2] for i in dataPathLs])
IH_list = np.delete(IH_list, np.argwhere(IH_list=='IHM01'))
IH_list = np.delete(IH_list, np.argwhere(IH_list=='IHM02'))
# IH_list = np.delete(IH_list, np.argwhere(IH_list=='IH2007'))
print(IH_list)

for l in IH_list:
    path = []
    df1 = pd.DataFrame()
    for i in dataPathLs:
        if os.path.basename(i).split('_')[2] == l:
            path.append(i)
    for j in path:
        df = pd.read_csv(j)
        df = df[["TRADINGDATE", "TRADINGTIME", "SYMBOL", "TOTALVOLUME", "TOTALAMOUNT", "LASTPRICE", "BUYPRICE01", "BUYPRICE02",
         "BUYPRICE03", "BUYPRICE04","BUYPRICE05","BUYVOLUME01", "BUYVOLUME02", "BUYVOLUME03","BUYVOLUME04", "BUYVOLUME05",
         "SELLPRICE01", "SELLPRICE02", "SELLPRICE03", "SELLPRICE04", "SELLPRICE05", "SELLVOLUME01",  "SELLVOLUME02", 
         "SELLVOLUME03","SELLVOLUME04","SELLVOLUME05","OPENPRICE", "SETTLEPRICE"]]
        df["TRADINGTIME"] = pd.to_datetime(df["TRADINGTIME"])
        df = df.rename(columns={"TRADINGDATE":"date", "TRADINGTIME":"time", "SYMBOL":"ID", "TOTALVOLUME":"cum_volume", "TOTALAMOUNT":"cum_amount",
                       "LASTPRICE":"close", "BUYPRICE01":"bid1p", "BUYPRICE02":"bid2p", "BUYPRICE03":"bid3p", "BUYPRICE04":"bid4p",
                       "BUYPRICE05":"bid5p", "BUYVOLUME01":"bid1q", "BUYVOLUME02":"bid2q", "BUYVOLUME03":"bid3q", "BUYVOLUME04":"bid4q",
                       "BUYVOLUME05":"bid5q", "SELLPRICE01":"ask1p", "SELLPRICE02":"ask2p", "SELLPRICE03":"ask3p", "SELLPRICE04":"ask4p",
                       "SELLPRICE05":"ask5p", "SELLVOLUME01": "ask1q", "SELLVOLUME02": "ask2q", "SELLVOLUME03": "ask3q", 
                       "SELLVOLUME04": "ask4q", "SELLVOLUME05": "ask5q", "OPENPRICE":"openPrice", "SETTLEPRICE":"settlePrice"})
        for i in ["cum_volume", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "ask1q", "ask2q", "ask3q","ask4q", "ask5q"]:
            df[i] = df[i].astype(int)
        df1 = pd.concat([df1, df]).reset_index(drop=True)
        print(j)
        print(df1.shape[0])
    
    d = kk[kk["ID"] == int(df1["ID"].iloc[0][2:])]["expDate"].values[0]
    df1["dayToExpiry"] = (d - pd.to_datetime(df1["date"].astype(str), format='%Y%m%d')).apply(lambda x: x.days)
    df1 = pd.merge(df1, s, on=["date", "ID"])
    print(df1.shape[0])
    try:
        ndf = pd.read_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')
        print(ndf.shape[0])
        print(ndf['date'].max())
        ndf = ndf[ndf['date'] < df1['date'].min()]
        assert(ndf['date'].max() < df1['date'].min())
        print(ndf.shape[0])
        df1 = pd.concat([ndf, df1]).reset_index(drop=True)
        print(df1.shape[0])
        print(df1["date"].min())
        print(df1["date"].max())
        df1.to_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')
    except:
        print('new contract')
        print(df1["date"].min())
        print(df1["date"].max())
        df1.to_pickle('\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\CFFEX 2018-2020\\' + l + '.pkl')

['IH2010' 'IH2011' 'IH2012' 'IH2101' 'IH2103' 'IH2106']
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202010\FFL2_TAQ_IH2010_202010.csv
128360
128360
626485
20201016
498125
626485
20200824
20201016
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202010\FFL2_TAQ_IH2011_202010.csv
343267
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202011\FFL2_TAQ_IH2011_202011.csv
687742
687742
280571
20201022
85961
773703
20200921
20201120
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202010\FFL2_TAQ_IH2012_202010.csv
279300
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202011\FFL2_TAQ_IH2012_202011.csv
732482
732482
1677788
20201022
1511607
2244089
20200420
20201130
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202011\FFL2_TAQ_IH2101_202011.csv
61511
61511
new contract
20201123
20201130
\\192.168.10.30\Kevin_zhenyu\cffex\CFFEX 2018-2020\IHdata_zip\IH_202010\FFL2_TAQ_IH2103_202010.csv
181784